# 고용산재보험 가입 현황 적용

2일차에서 `pandas`패키지의 `DataFrame`구조의 데이터에 대해 `merge` 및 `group`함수를 이용하여 데이터를 조작하는 법을 배웠다. 이때, 실제 데이터가 아닌 완성된 데이터의 일부를 이용하여 진행했다. 이번 시간에는 raw data(아무것도 하지 않은 데이터)를 이용하여 원하는 결과를 확인하는 과정을 가지고자 한다.

이번 시간의 목표는 2일차 때 사용한 데이터를 고용산재보험 가입 현황데이터를 이용하여 생성하는 것이다. 목표는 다음과 같다.

2022년 제외하고, 
1. 2010년 이후 산재 및 고용보험 성립 연도별, 시도별 평균 근로자수 및 가입된 사업장 수 계산
2. 2017년 이후 산재 및 고용보험 성립 연도별, 시도별 음식점에 대한 사업장 수 계산

In [1]:
import pandas as pd

## 고용산재보험 데이터

데이터들은 각각 서울, 대구, 대전, 부산, 서울, 울산, 인천에 대해 csv파일이 각각 저장되어 있다. 이들을 모두 읽고 확인해본다.

In [2]:
gj_dat = pd.read_csv("https://uos-bigdata.github.io/lab_data/docs/assets/data_lab_bokji/dat_practice_gwangju.csv")
dg_dat = pd.read_csv("https://uos-bigdata.github.io/lab_data/docs/assets/data_lab_bokji/dat_practice_daegu.csv")
dj_dat = pd.read_csv("https://uos-bigdata.github.io/lab_data/docs/assets/data_lab_bokji/dat_practice_daejeon.csv")
bs_dat = pd.read_csv("https://uos-bigdata.github.io/lab_data/docs/assets/data_lab_bokji/dat_practice_pusan.csv")
sl_dat = pd.read_csv("https://uos-bigdata.github.io/lab_data/docs/assets/data_lab_bokji/dat_practice_seoul.csv")
us_dat = pd.read_csv("https://uos-bigdata.github.io/lab_data/docs/assets/data_lab_bokji/dat_practice_ulsan.csv")
ic_dat = pd.read_csv("https://uos-bigdata.github.io/lab_data/docs/assets/data_lab_bokji/dat_practice_incheon.csv")

In [3]:
sl_dat.head()

,연번,보험구분,사업장명,우편번호,사업장주소,고용보험업종코드,고용보험업종명,산재보험성립일자,고용보험성립일자,산재보험상시근로자수,고용보험상시근로자수,산재사업구분,고용사업구분,사업자등록번호,시도
0,1,전체,김구동(부동산),7238,서울 영등포구 국회대로72길 11,74100.0,사업시설 유지관리 서비스업,2010-07-01,2010-07-01,1,1,계속,계속,1.071384e+09,서울
1,2,전체,바우피엔피,4546,서울 중구 을지로33길 16,20202.0,합성수지 및 기타 플라스틱 물질 제조업,2014-01-01,2014-01-01,2,2,계속,계속,1.010109e+09,서울
2,3,전체,토탈사인대광부식,7291,서울특별시 영등포구 영등포로18길 2-3,18119.0,기타 인쇄업,2008-04-01,2008-04-01,3,3,계속,계속,1.010113e+09,서울
3,5,전체,한서전자,4545,서울 중구 청계천로 160,46521.0,가전제품 및 부품 도매업,2000-07-01,1998-10-01,5,5,계속,계속,1.010113e+09,서울
4,6,전체,삼진상사,8217,서울 구로구 경인로53길 15,46595.0,전기용 기계장비 및 관련 기자재 도매업,2000-07-01,1998-10-01,1,1,계속,계속,1.010113e+09,서울


이들은 각각 `DataFrame`으로 저장되어 있다. `pd.concat`함수를 이용해서 row를 기준으로 합치는 작업을 진행한다.

이때, `pandas` 패키지 내에 `concat`함수를 이용하면 된다. 이는 `merge`랑 다른 의미의 함수이다.

In [4]:
tmp_dat = pd.concat([gj_dat, dg_dat, dj_dat, bs_dat, sl_dat, us_dat, ic_dat])

In [5]:
tmp_dat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1066598 entries, 0 to 117092
Data columns (total 15 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   연번          1066598 non-null  int64  
 1   보험구분        1066598 non-null  object 
 2   사업장명        1066597 non-null  object 
 3   우편번호        1066598 non-null  int64  
 4   사업장주소       1066598 non-null  object 
 5   고용보험업종코드    1032051 non-null  float64
 6   고용보험업종명     1032051 non-null  object 
 7   산재보험성립일자    1060177 non-null  object 
 8   고용보험성립일자    1032047 non-null  object 
 9   산재보험상시근로자수  1066598 non-null  int64  
 10  고용보험상시근로자수  1066598 non-null  int64  
 11  산재사업구분      1060177 non-null  object 
 12  고용사업구분      1032056 non-null  object 
 13  사업자등록번호     1066412 non-null  object 
 14  시도          1066598 non-null  object 
dtypes: float64(1), int64(4), object(10)
memory usage: 130.2+ MB


In [6]:
tmp_dat.head()

,연번,보험구분,사업장명,우편번호,사업장주소,고용보험업종코드,고용보험업종명,산재보험성립일자,고용보험성립일자,산재보험상시근로자수,고용보험상시근로자수,산재사업구분,고용사업구분,사업자등록번호,시도
0,696,전체,남해상사,12724,경기 광주시 곤지암읍 새재길190번길 31-14,13999.0,그 외 기타 분류 안된 섬유제품 제조업,2014-06-15,2014-06-15,1,1,계속,계속,1010366494.0,광주
1,1279,전체,MK스타일,12817,경기도 광주시 도척면 도척로699번길30-61,13221.0,침구 및 관련제품 제조업,2013-04-01,2013-04-01,2,2,계속,계속,1010614643.0,광주
2,1597,전체,무등중기,62073,광주 서구 매월2로15번길 16,42600.0,건설장비 운영업,2020-08-26,2020-08-26,1,1,계속,계속,1010711172.0,광주
3,1615,전체,아이투비,12787,경기 광주시 태봉로 110,23221.0,가정용 및 장식용 도자기 제조업,2012-01-12,2012-01-12,2,2,계속,계속,1010715412.0,광주
4,2185,전체,바우,12769,경기 광주시 광남안로 483,46742.0,직물 도매업,2010-04-01,2010-04-01,1,1,계속,계속,1010887591.0,광주


사업자등록번호가 문자형으로 지정되어 있는 것을 알 수 있다. 즉, 사업자등록번호 중 몇개의 값이 문자의 형태로 저장되었다는 것을 의미한다.

## 산재 및 고용보험 성립 연도별, 시도별 평균 근로자수, 전체 사업장 수, 가입되지 않은 사업장 수 계산

이 작업을 하기 위해서는 총 2가지 작업을 진행해야 한다.

1. 산재 관련 변수들의 연도별, 시도별 시도별 평균 근로자수, 전체 사업장 수, 가입되지 않은 사업장 수를 계산
2. 고용 관련 변수들의 연도별, 시도별 시도별 평균 근로자수, 전체 사업장 수, 가입되지 않은 사업장 수를 계산
3. (1.), (2.)에서 생성한 데이터를 결합

(1.), (2.)을 구분하는 이유는 산재보험성립일자 및 고용보험성립일자의 연도가 반드시 같다는 보장이 없으며, 가입되지 않은 것도 제각각이기 때문이다. 먼저 산재 관련한 데이터 셋을 생성할건데, 데이터에 연도 관련 변수가 없으므로, 먼저 연도를 생성하자.

산재보험성립일자에서 연도를 생성하는 방법은 2가지이다. 
1) 앞에서 4자리를 자르는것
2) 이 변수를 날짜형으로 저장하고, 연도만 추출하는것

우리는 두번째 방법을 이용하여 연도를 생성한다. 날짜형으로 저장하기 위해 `pandas`패키지안에 `to_datetime` method를 이용한다.

In [7]:
tmp_dat['산재보험성립일자'] = pd.to_datetime(tmp_dat['산재보험성립일자'])

In [8]:
tmp_dat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1066598 entries, 0 to 117092
Data columns (total 15 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   연번          1066598 non-null  int64         
 1   보험구분        1066598 non-null  object        
 2   사업장명        1066597 non-null  object        
 3   우편번호        1066598 non-null  int64         
 4   사업장주소       1066598 non-null  object        
 5   고용보험업종코드    1032051 non-null  float64       
 6   고용보험업종명     1032051 non-null  object        
 7   산재보험성립일자    1060177 non-null  datetime64[ns]
 8   고용보험성립일자    1032047 non-null  object        
 9   산재보험상시근로자수  1066598 non-null  int64         
 10  고용보험상시근로자수  1066598 non-null  int64         
 11  산재사업구분      1060177 non-null  object        
 12  고용사업구분      1032056 non-null  object        
 13  사업자등록번호     1066412 non-null  object        
 14  시도          1066598 non-null  object        
dtypes: datetime64[ns](1), float64(1),

`산재보험성립일자`의 Dtype을 보면 처음 읽었을때랑 다르게 datetime64로 변경 되었음을 확인 할 수 있다.

다음으로 연도를 추출하기 위해 `dt` method의 `year`함수를 이용한다.

In [9]:
tmp_dat["산재_year"] = tmp_dat["산재보험성립일자"].dt.year

tmp_dat.head()

,연번,보험구분,사업장명,우편번호,사업장주소,고용보험업종코드,고용보험업종명,산재보험성립일자,고용보험성립일자,산재보험상시근로자수,고용보험상시근로자수,산재사업구분,고용사업구분,사업자등록번호,시도,산재_year
0,696,전체,남해상사,12724,경기 광주시 곤지암읍 새재길190번길 31-14,13999.0,그 외 기타 분류 안된 섬유제품 제조업,2014-06-15,2014-06-15,1,1,계속,계속,1010366494.0,광주,2014.0
1,1279,전체,MK스타일,12817,경기도 광주시 도척면 도척로699번길30-61,13221.0,침구 및 관련제품 제조업,2013-04-01,2013-04-01,2,2,계속,계속,1010614643.0,광주,2013.0
2,1597,전체,무등중기,62073,광주 서구 매월2로15번길 16,42600.0,건설장비 운영업,2020-08-26,2020-08-26,1,1,계속,계속,1010711172.0,광주,2020.0
3,1615,전체,아이투비,12787,경기 광주시 태봉로 110,23221.0,가정용 및 장식용 도자기 제조업,2012-01-12,2012-01-12,2,2,계속,계속,1010715412.0,광주,2012.0
4,2185,전체,바우,12769,경기 광주시 광남안로 483,46742.0,직물 도매업,2010-04-01,2010-04-01,1,1,계속,계속,1010887591.0,광주,2010.0


같은 방법으로 `고용보험성립일자`의 연도를 생성한다.

In [10]:
tmp_dat['고용보험성립일자'] = pd.to_datetime(tmp_dat['고용보험성립일자'])
tmp_dat["고용_year"] = tmp_dat["고용보험성립일자"].dt.year

tmp_dat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1066598 entries, 0 to 117092
Data columns (total 17 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   연번          1066598 non-null  int64         
 1   보험구분        1066598 non-null  object        
 2   사업장명        1066597 non-null  object        
 3   우편번호        1066598 non-null  int64         
 4   사업장주소       1066598 non-null  object        
 5   고용보험업종코드    1032051 non-null  float64       
 6   고용보험업종명     1032051 non-null  object        
 7   산재보험성립일자    1060177 non-null  datetime64[ns]
 8   고용보험성립일자    1032047 non-null  datetime64[ns]
 9   산재보험상시근로자수  1066598 non-null  int64         
 10  고용보험상시근로자수  1066598 non-null  int64         
 11  산재사업구분      1060177 non-null  object        
 12  고용사업구분      1032056 non-null  object        
 13  사업자등록번호     1066412 non-null  object        
 14  시도          1066598 non-null  object        
 15  산재_year     1060177 non-null  flo

산재보험성립일자 및 고용보험성립일자의 결측치의 갯수는 다음과 같다.

In [11]:
[tmp_dat["산재보험성립일자"].isna().sum(), tmp_dat["고용보험성립일자"].isna().sum()]

[6421, 34551]

각각 2632개, 32963개의 결측값을 포함하고 있다. 그러므로, 연도별, 시도별로 계산하기 위해, boolean변수를 생성한다.

In [12]:
tmp_dat["산재_bool"] = tmp_dat["산재보험성립일자"].isna()
tmp_dat["고용_bool"] = tmp_dat["고용보험성립일자"].isna()

tmp_dat.head()

,연번,보험구분,사업장명,우편번호,사업장주소,고용보험업종코드,고용보험업종명,산재보험성립일자,고용보험성립일자,산재보험상시근로자수,고용보험상시근로자수,산재사업구분,고용사업구분,사업자등록번호,시도,산재_year,고용_year,산재_bool,고용_bool
0,696,전체,남해상사,12724,경기 광주시 곤지암읍 새재길190번길 31-14,13999.0,그 외 기타 분류 안된 섬유제품 제조업,2014-06-15,2014-06-15,1,1,계속,계속,1010366494.0,광주,2014.0,2014.0,False,False
1,1279,전체,MK스타일,12817,경기도 광주시 도척면 도척로699번길30-61,13221.0,침구 및 관련제품 제조업,2013-04-01,2013-04-01,2,2,계속,계속,1010614643.0,광주,2013.0,2013.0,False,False
2,1597,전체,무등중기,62073,광주 서구 매월2로15번길 16,42600.0,건설장비 운영업,2020-08-26,2020-08-26,1,1,계속,계속,1010711172.0,광주,2020.0,2020.0,False,False
3,1615,전체,아이투비,12787,경기 광주시 태봉로 110,23221.0,가정용 및 장식용 도자기 제조업,2012-01-12,2012-01-12,2,2,계속,계속,1010715412.0,광주,2012.0,2012.0,False,False
4,2185,전체,바우,12769,경기 광주시 광남안로 483,46742.0,직물 도매업,2010-04-01,2010-04-01,1,1,계속,계속,1010887591.0,광주,2010.0,2010.0,False,False


### 산재 기준 데이터 생성

산재 기준으로 데이터를 생성해보자. 먼저 연도별, 시도별로 산재보험상시근로자수를 계산해보자.

먼저, 필요한 변수 선택 및 생성한다. 이때, 산재_year가 결측값이면, 산재_bool도 결측값이 생성된다. 그러므로, 고용_bool에 대한 합을 계산하여 산재보험은 가입했지만, 고용보험은 가입하지 않은 수를 계산한다.

In [13]:
san_dat = tmp_dat.loc[:, ["산재_year", "시도", "산재보험상시근로자수", "고용_bool"]]
san_dat["count"] = 1

다음으로, `groupby` method와 `agg` method를 이용하여 산재보험상시근로자수는 `mean`함수, count는 `sum`함수를 계산한다.

In [14]:
san_agg_dat = san_dat.groupby(["산재_year", "시도"]).agg({"산재보험상시근로자수" : "mean", "고용_bool" : "sum", "count" : "sum"})

`reset_index` method를 이용하여 데이터를 정리한다.

In [15]:
san_agg_dat = san_agg_dat.reset_index()
san_agg_dat

,산재_year,시도,산재보험상시근로자수,고용_bool,count
0,1960.0,부산,25.000000,0,1
1,1960.0,서울,16.000000,0,1
2,1960.0,인천,38.000000,1,1
3,1962.0,부산,61.000000,0,1
4,1964.0,광주,165.000000,0,1
...,...,...,...,...,...
406,2021.0,인천,2.604997,703,15648
407,2022.0,광주,1.000000,0,1
408,2022.0,대구,1.000000,1,1
409,2022.0,대전,6.500000,2,4


이제, 2010년 이상, 2021년 이하의 데이터를 추출한다. 이번 예시에서는 해당사항이 아니지만, 주의해야 할 점은 결측값(`NaN`)은 어떤 비교연산자를 이용하더라도 모두 `False`를 출력한다는 것이다.

In [16]:
tmp_ind = (san_agg_dat["산재_year"] >= 2010) & (san_agg_dat["산재_year"] <= 2021)
san_agg_dat = san_agg_dat.loc[tmp_ind, :]

san_agg_dat

,산재_year,시도,산재보험상시근로자수,고용_bool,count
323,2010.0,광주,6.863187,52,1820
324,2010.0,대구,7.366627,82,2523
325,2010.0,대전,9.100746,59,1340
326,2010.0,부산,8.126126,121,3108
327,2010.0,서울,10.971553,343,11741
...,...,...,...,...,...
402,2021.0,대전,2.658709,434,7917
403,2021.0,부산,2.874647,688,17367
404,2021.0,서울,7.487265,2375,59009
405,2021.0,울산,3.442591,273,5095


마지막으로 변수명들을 정리하자.

In [17]:
san_agg_dat.columns = ["연도", "시도", "산재_평균근로자수", "산재만_가입된사업장수", "산재_가입된사업장수"]
san_agg_dat

,연도,시도,산재_평균근로자수,산재만_가입된사업장수,산재_가입된사업장수
323,2010.0,광주,6.863187,52,1820
324,2010.0,대구,7.366627,82,2523
325,2010.0,대전,9.100746,59,1340
326,2010.0,부산,8.126126,121,3108
327,2010.0,서울,10.971553,343,11741
...,...,...,...,...,...
402,2021.0,대전,2.658709,434,7917
403,2021.0,부산,2.874647,688,17367
404,2021.0,서울,7.487265,2375,59009
405,2021.0,울산,3.442591,273,5095


여기서, 산재만_가입된사업장수의 의미는 연도, 시도별로 산재가 가입된 연도에, 고용보험이 가입되지 않은 사업장 수를 의미하는 것이다.

### 고용 기준 데이터 생성

위에 산재 기준을 생성시킨 데이터를 고용 기준으로 데이터를 생성한다. 산재 데이터 생성과 같은 방식으로 생성한다.

먼저, 필요 변수 선택 및 생성을 진행한다. 마찬가지로, 고용_year가 결측값이면, 고용_bool도 결측값이 생성된다. 그러므로, 산재_bool에 대한 합을 계산하여 고용보험은 가입했지만, 산재보험은 가입하지 않은 수를 계산한다.

In [18]:
go_dat = tmp_dat.loc[:, ["고용_year", "시도", "고용보험상시근로자수", "산재_bool"]]
go_dat["count"] = 1

다음으로, `groupby` method와 `agg` method를 이용하여 고용보험상시근로자수는 `mean`함수, count는 `sum`함수를 계산한다.

In [19]:
go_agg_dat = go_dat.groupby(["고용_year", "시도"]).agg({"고용보험상시근로자수" : "mean", "산재_bool" : sum, "count" : "sum"})

`reset_index` method를 이용하여 데이터를 정리한다.

In [20]:
go_agg_dat = go_agg_dat.reset_index()
go_agg_dat

,고용_year,시도,고용보험상시근로자수,산재_bool,count
0,1977.0,서울,0.000000,0,1
1,1978.0,서울,1.000000,0,1
2,1981.0,서울,1.000000,0,1
3,1989.0,부산,18.000000,0,1
4,1991.0,서울,16.000000,0,1
...,...,...,...,...,...
193,2021.0,울산,2.080108,262,5193
194,2021.0,인천,1.916362,444,15615
195,2022.0,광주,0.500000,0,2
196,2022.0,대전,12.000000,0,2


2010년 이상, 2020년 이하의 데이터를 추출한다.

In [21]:
tmp_ind = (go_agg_dat["고용_year"] >= 2010) & (go_agg_dat['고용_year'] <= 2021)
go_agg_dat = go_agg_dat.loc[tmp_ind, :]

go_agg_dat

,고용_year,시도,고용보험상시근로자수,산재_bool,count
111,2010.0,광주,6.847419,1,1763
112,2010.0,대구,7.051240,7,2420
113,2010.0,대전,8.816680,3,1271
114,2010.0,부산,7.622692,11,2979
115,2010.0,서울,10.869985,30,11391
...,...,...,...,...,...
190,2021.0,대전,1.887973,369,8007
191,2021.0,부산,1.909389,660,17680
192,2021.0,서울,2.020801,1410,58700
193,2021.0,울산,2.080108,262,5193


In [22]:
go_agg_dat.columns = ["연도", "시도", "고용_평균근로자수", "고용만_가입된사업장수", "고용_가입된사업장수"]
go_agg_dat

,연도,시도,고용_평균근로자수,고용만_가입된사업장수,고용_가입된사업장수
111,2010.0,광주,6.847419,1,1763
112,2010.0,대구,7.051240,7,2420
113,2010.0,대전,8.816680,3,1271
114,2010.0,부산,7.622692,11,2979
115,2010.0,서울,10.869985,30,11391
...,...,...,...,...,...
190,2021.0,대전,1.887973,369,8007
191,2021.0,부산,1.909389,660,17680
192,2021.0,서울,2.020801,1410,58700
193,2021.0,울산,2.080108,262,5193


여기서, 고용만_가입된사업장수의 의미는 각 연도, 시도별로 고용이 가입된 연도에, 산재보험이 가입되지 않은 사업장 수를 의미하는 것이다.

마지막으로, 두 데이터셋을 `merge` 함수를 이용하여 결합한다.

In [23]:
join_dat = pd.merge(san_agg_dat, go_agg_dat, on = ["연도", "시도"])

join_dat

,연도,시도,산재_평균근로자수,산재만_가입된사업장수,산재_가입된사업장수,고용_평균근로자수,고용만_가입된사업장수,고용_가입된사업장수
0,2010.0,광주,6.863187,52,1820,6.847419,1,1763
1,2010.0,대구,7.366627,82,2523,7.051240,7,2420
2,2010.0,대전,9.100746,59,1340,8.816680,3,1271
3,2010.0,부산,8.126126,121,3108,7.622692,11,2979
4,2010.0,서울,10.971553,343,11741,10.869985,30,11391
...,...,...,...,...,...,...,...,...
79,2021.0,대전,2.658709,434,7917,1.887973,369,8007
80,2021.0,부산,2.874647,688,17367,1.909389,660,17680
81,2021.0,서울,7.487265,2375,59009,2.020801,1410,58700
82,2021.0,울산,3.442591,273,5095,2.080108,262,5193


In [24]:
join_dat.to_csv("../전체_사업장.csv", index = False)

## 연도별 음식점에 대한 가입된 사업장의 수

음식점만 집중적으로 살펴봤을때, 산재 및 고용보험은 일반적으로 가게를 새로 열거나, 새로운 사람 혹은 아르바이트 생을 뽑았을때 가입을 할 것이다. 그렇다면, 코로나 집중기간이였던 2020년에는 과연 가입된 사업장 수가 감소했을까??? 비교 대상으로는 2016년부터 2019년까지 4년치의 데이터를 비교한다.

이를 얻기 위해서는 음식점에 대한 데이터를 생성하여, 연도별, 시도별로 가입된 사업장수를 살펴보고자 한다. 그러기 위해서는 고용보험업종코드 앞의 2자리가 `56`인 데이터를 가져와야 한다. 따라서, 고용보험업종코드에서 앞의 2자리를 저장하고, `56`만 가져오는 작업을 진행한다.

그러기 위해 `str.slice()`를 사용할 건데, 이는 `DataFrame`에서 하나의 column 혹은 row에 대해 일부를 자르고 싶을때 사용하는 method이다. 또한, 이 method는 문자열에 적용된다.

먼저, 고용보험업종코드는 숫자형이므로 `astype` method를 이용해서 문자형으로 변형을 시켜준다.
이후, `str.slice` method를 이용하여 앞의 2자리를 자른다.

In [25]:
tmp_ser = tmp_dat["고용보험업종코드"].astype(str)
tmp_dat["부분_고용보험업종코드"] = tmp_ser.str.slice(0, 2)

tmp_dat.head()

,연번,보험구분,사업장명,우편번호,사업장주소,고용보험업종코드,고용보험업종명,산재보험성립일자,고용보험성립일자,산재보험상시근로자수,고용보험상시근로자수,산재사업구분,고용사업구분,사업자등록번호,시도,산재_year,고용_year,산재_bool,고용_bool,부분_고용보험업종코드
0,696,전체,남해상사,12724,경기 광주시 곤지암읍 새재길190번길 31-14,13999.0,그 외 기타 분류 안된 섬유제품 제조업,2014-06-15,2014-06-15,1,1,계속,계속,1010366494.0,광주,2014.0,2014.0,False,False,13
1,1279,전체,MK스타일,12817,경기도 광주시 도척면 도척로699번길30-61,13221.0,침구 및 관련제품 제조업,2013-04-01,2013-04-01,2,2,계속,계속,1010614643.0,광주,2013.0,2013.0,False,False,13
2,1597,전체,무등중기,62073,광주 서구 매월2로15번길 16,42600.0,건설장비 운영업,2020-08-26,2020-08-26,1,1,계속,계속,1010711172.0,광주,2020.0,2020.0,False,False,42
3,1615,전체,아이투비,12787,경기 광주시 태봉로 110,23221.0,가정용 및 장식용 도자기 제조업,2012-01-12,2012-01-12,2,2,계속,계속,1010715412.0,광주,2012.0,2012.0,False,False,23
4,2185,전체,바우,12769,경기 광주시 광남안로 483,46742.0,직물 도매업,2010-04-01,2010-04-01,1,1,계속,계속,1010887591.0,광주,2010.0,2010.0,False,False,46


그리고 생성된 변수가 문자로 `56`의 값을 가지는 데이터만 불러오자.

In [26]:
tmp_ind = tmp_dat["부분_고용보험업종코드"] == "56"
sub_dat = tmp_dat.loc[tmp_ind, :]

sub_dat.head()

,연번,보험구분,사업장명,우편번호,사업장주소,고용보험업종코드,고용보험업종명,산재보험성립일자,고용보험성립일자,산재보험상시근로자수,고용보험상시근로자수,산재사업구분,고용사업구분,사업자등록번호,시도,산재_year,고용_year,산재_bool,고용_bool,부분_고용보험업종코드
13,4422,전체,돈가네김치찌개곤지암점,12804,경기 광주시 곤지암읍 경충대로 571,56111.0,한식 일반 음식점업,2020-01-28,2020-01-28,2,2,계속,계속,1011562623.0,광주,2020.0,2020.0,False,False,56
16,4470,전체,윤장어,12768,경기 광주시 순암로 180,56114.0,한식 해산물요리 전문점,2020-08-17,2020-08-17,1,1,계속,계속,1011613681.0,광주,2020.0,2020.0,False,False,56
20,4527,전체,무진장해물탕,62319,광주 광산구 목련로273번안길 61,56111.0,한식 일반 음식점업,2019-01-01,2019-01-01,0,0,계속,계속,1011693313.0,광주,2019.0,2019.0,False,False,56
32,4986,전체,디저트39 광주상무점,61950,광주 서구 치평로 86,56221.0,커피 전문점,2019-07-01,2019-07-01,1,1,계속,계속,1012566327.0,광주,2019.0,2019.0,False,False,56
33,4990,전체,이비가짬뽕(전남대점),61220,광주 북구 용봉로 10,56121.0,중식 음식점업,2015-09-01,2015-09-01,16,16,계속,계속,1012572872.0,광주,2015.0,2015.0,False,False,56


이제 연도별, 시도별로 사업장의 갯수를 계산한다. 이때도 마찬가지로 `count`변수를 생성해 `sido`, `산재_year`, `고용_year`별로 계산한다.~~

In [27]:
sub_dat["count"] = 1
san_agg_dat = sub_dat.groupby(["시도", "산재_year"])["count"].agg("sum")

san_agg_dat

C:\Users\jieun\AppData\Local\Temp\ipykernel_36584\4066125944.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_dat["count"] = 1


시도  산재_year
광주  1988.0        1
    1999.0        1
    2000.0       40
    2001.0        4
    2002.0       10
               ... 
인천  2017.0     1319
    2018.0     1817
    2019.0     2718
    2020.0     2886
    2021.0     2891
Name: count, Length: 209, dtype: int64

In [28]:
san_agg_dat = san_agg_dat.reset_index()

san_agg_dat

,시도,산재_year,count
0,광주,1988.0,1
1,광주,1999.0,1
2,광주,2000.0,40
3,광주,2001.0,4
4,광주,2002.0,10
...,...,...,...
204,인천,2017.0,1319
205,인천,2018.0,1817
206,인천,2019.0,2718
207,인천,2020.0,2886


In [29]:
tmp_ind = (san_agg_dat["산재_year"] >= 2017) & (san_agg_dat['산재_year'] <= 2021)
san_agg_dat = san_agg_dat.loc[tmp_ind, :]

san_agg_dat.head()

,시도,산재_year,count
19,광주,2017.0,796
20,광주,2018.0,1121
21,광주,2019.0,1674
22,광주,2020.0,1757
23,광주,2021.0,1727


In [30]:
san_agg_dat.columns = ["시도", "연도", "산재_음식사업장수"]

san_agg_dat.head()

,시도,연도,산재_음식사업장수
19,광주,2017.0,796
20,광주,2018.0,1121
21,광주,2019.0,1674
22,광주,2020.0,1757
23,광주,2021.0,1727


같은 형태로, 고용에 대해서 진행한다.

In [31]:
sub_dat["count"] = 1
go_agg_dat = sub_dat.groupby(["시도", "고용_year"])["count"].agg("sum")

go_agg_dat

C:\Users\jieun\AppData\Local\Temp\ipykernel_36584\2061023033.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_dat["count"] = 1


시도  고용_year
광주  1998.0       27
    1999.0        3
    2000.0       12
    2001.0        4
    2002.0       10
               ... 
인천  2017.0     1318
    2018.0     1835
    2019.0     2731
    2020.0     2892
    2021.0     2898
Name: count, Length: 177, dtype: int64

In [32]:
go_agg_dat = go_agg_dat.reset_index()

go_agg_dat

,시도,고용_year,count
0,광주,1998.0,27
1,광주,1999.0,3
2,광주,2000.0,12
3,광주,2001.0,4
4,광주,2002.0,10
...,...,...,...
172,인천,2017.0,1318
173,인천,2018.0,1835
174,인천,2019.0,2731
175,인천,2020.0,2892


In [33]:
tmp_ind = (go_agg_dat["고용_year"] >= 2017) & (go_agg_dat['고용_year'] <= 2021)
go_agg_dat = go_agg_dat.loc[tmp_ind, :]

go_agg_dat.head()

,시도,고용_year,count
19,광주,2017.0,801
20,광주,2018.0,1136
21,광주,2019.0,1679
22,광주,2020.0,1763
23,광주,2021.0,1734


In [34]:
go_agg_dat.columns = ["시도", "연도", "고용_음식사업장수"]

go_agg_dat.head()

,시도,연도,고용_음식사업장수
19,광주,2017.0,801
20,광주,2018.0,1136
21,광주,2019.0,1679
22,광주,2020.0,1763
23,광주,2021.0,1734


두 데이터셋을 시도, 연도 기준으로 `merge`함수를 이용해서 합치자.

In [35]:
join_dat = pd.merge(san_agg_dat, go_agg_dat, on = ["시도", "연도"])

join_dat

,시도,연도,산재_음식사업장수,고용_음식사업장수
0,광주,2017.0,796,801
1,광주,2018.0,1121,1136
2,광주,2019.0,1674,1679
3,광주,2020.0,1757,1763
4,광주,2021.0,1727,1734
5,대구,2017.0,1367,1375
6,대구,2018.0,1739,1749
7,대구,2019.0,2523,2531
8,대구,2020.0,2428,2434
9,대구,2021.0,2619,2624


In [36]:
join_dat.to_csv("../음식_사업장.csv", index = False)

이렇게 1일차, 2일차에 사용한 데이터를 생성 할 수 있다.